In [1]:
!pip install kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [2]:
!kaggle datasets download -d duyminhnguyentran/csc15105

100% 9.60M/9.60M [00:00<00:00, 99.6MB/s]
100% 9.60M/9.60M [00:00<00:00, 99.0MB/s]


In [3]:
!unzip -q csc15105.zip -d /content/

In [4]:
import pandas as pd

data = pd.read_json("Project1_Data.json")
data

,id,question,title,text,label
0,u7-1570446247_1,Quang Hải giành được chức vô địch U21 quốc gia...,Nguyễn Quang Hải (sinh 1997),"Năm 2013 , Nguyễn Quang Hải giành chức vô địch...",True
1,u7-1570446247_2,Quang Hải giành được chức vô địch U21 quốc gia...,Nguyễn Quang Hải (sinh 1997),"Sau chức vô địch U-21 quốc gia 2013 , Nguyễn Q...",True
2,u7-1570446247_0,Quang Hải giành được chức vô địch U21 quốc gia...,Nguyễn Quang Hải (sinh 1997),Anh bắt đầu gia nhập lò đào tạo trẻ Hà Nội T&T...,False
3,u7-1570446247_3,Quang Hải giành được chức vô địch U21 quốc gia...,Nguyễn Quang Hải (sinh 1997),"Năm 2011 , Nguyễn Quang Hải mới 14 tuổi được g...",False
4,u7-1570445661_0,Mỗi hiệp bóng đá kéo dài bao lâu,Bóng đá,Một trận đấu bóng đá thông thường có hai hiệp ...,True
...,...,...,...,...,...
18103,u6-1551268099_0,Thành phố nào là thủ đô của Costa Rica,"San José, Costa Rica",San José (tiếng Tây Ban Nha: San José; ) là th...,True
18104,u6-1551268099_1,Thành phố nào là thủ đô của Costa Rica,Costa Rica,Vì Costa Rica có địa hình đa dạng nhiều núi no...,True
18105,u6-1551268099_2,Thành phố nào là thủ đô của Costa Rica,"San José, Costa Rica","Với địa vị là thủ đô, San José là nơi đặt trụ ...",True
18106,u6-1551268099_4,Thành phố nào là thủ đô của Costa Rica,"Santa Teresa, Costa Rica",Santa Teresa là một thị xã nhỏ ở tỉnh Puntaren...,True


In [105]:
import torch
import torch.nn as nn
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from transformers import AutoModel, AutoTokenizer
import torch.optim as optim
import logging

logging.getLogger("transformers.tokenization_utils_base").setLevel(logging.ERROR)

class BARTPhoEmbedding(nn.Module):
    def __init__(self, pretrained_model="vinai/bartpho-syllable"):
        super(BARTPhoEmbedding, self).__init__()
        self.bartpho_model = AutoModel.from_pretrained(pretrained_model)
        self.tokenizer = AutoTokenizer.from_pretrained(pretrained_model)

    def forward(self, input_text):
        input_ids = self.tokenizer(input_text, return_tensors="pt")["input_ids"]
        outputs = self.bartpho_model(input_ids)
        return outputs.hidden_state

class TextClassifier(nn.Module):
    def __init__(self, embedding_model, num_classes):
        super(TextClassifier, self).__init__()

        # Extract embedding dimension from the embedding_model
        embedding_dim = embedding_model.bartpho_model.config.hidden_size

        # Update the number of input channels in Conv1d layers
        self.convs = nn.ModuleList([
            nn.Conv1d(32, 128, kernel_size=3, stride=2),
            nn.Conv1d(32, 128, kernel_size=3, stride=2),
            nn.Conv1d(32, 128, kernel_size=3, stride=2),
        ])

        # Fully connected layer
        self.fc = nn.Linear(384, 32)

        # Dropout
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        # Apply convolution and max pooling
        x = [torch.relu(conv(x.float())).squeeze(-1) for conv in self.convs]
        x = [nn.functional.max_pool1d(conv, conv.size(-1)).squeeze(-1) for conv in x]

        # Concatenate features
        x = torch.cat(x, dim=-1)

        # Fully connected layer
        x = self.dropout(x)
        x = self.fc(x)

        return x


class VNQADataset(Dataset):
    def __init__(self, x, y, tokenizer, max_length=128):
        self.x = x
        self.y = y
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.x)

    def __getitem__(self, idx):
        sample = self.x.iloc[idx]

        question = sample["question"]
        text = sample["text"]
        label = self.y.iloc[idx]

        encoding = self.tokenizer(question, text, padding='max_length', truncation=True, max_length=self.max_length, return_tensors='pt')

        if 'input_ids' in encoding and 'attention_mask' in encoding:
            input_ids = encoding['input_ids'].squeeze()
            attention_mask = encoding['attention_mask'].squeeze()

            result = {'input_ids': input_ids, 'attention_mask': attention_mask, 'label': label} if label is not None else {}
            # print(f"After Condition - Result: {result}")
            return result

        print("Empty Encoding")
        return {}

def train_and_evaluate(model, train_dataloader, test_dataloader, criterion, optimizer, num_epochs):
    for epoch in range(num_epochs):
        print("Epoch -", epoch + 1)

        # Training
        model.train()
        for batch in train_dataloader:
            if not batch:
                continue
            question, text, label = batch['input_ids'], batch['attention_mask'], batch['label']

            optimizer.zero_grad()

            question = question.long()
            text = text.long()

            # Concatenate tensors along dim=1
            combined_input = torch.cat((question, text), dim=1)
            try:
                output = model(combined_input)
                output = output.squeeze()

                if label is not None:
                    loss = criterion(output.float(), label.float())
                    loss.backward()
                    optimizer.step()
            except RuntimeError as e:
                if "to have 32 channels" in str(e):
                    print("")
                else:
                    raise e
        # Evaluation
        model.eval()
        with torch.no_grad():
            correct = 0
            total = 0
            for batch in test_dataloader:
                if not batch:
                    continue
                question, text, label = batch['input_ids'], batch['attention_mask'], batch['label']

                question = question.long()
                text = text.long()

                combined_input = torch.cat((question, text), dim=1)
                try:
                  output = model(combined_input)
                  output = output.squeeze()
                  output_reshaped = output.view(32, -1)

                  if label is not None:
                      _, predicted = torch.max(output_reshaped.data, 1)
                      total += label.size(0)
                      correct += (predicted == label).sum().item()
                except RuntimeError as e:
                  if "to have 32 channels" in str(e):
                      print("")
                  else:
                      raise e

        accuracy = correct / total if total > 0 else 0.0
        print(f"Epoch {epoch + 1}/{num_epochs}, Test Accuracy: {accuracy * 100:.2f}%")

data = pd.read_json("Project1_Data.json")
y = data["label"]
x = data.drop(["label", "id"], axis=1)

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=2024)

tokenizer = AutoTokenizer.from_pretrained("vinai/bartpho-syllable")

train_dataset = VNQADataset(x_train, y_train, tokenizer=tokenizer)
test_dataset = VNQADataset(x_test, y_test, tokenizer=tokenizer)
# for idx in range(min(5, len(test_dataset))):
#     sample = test_dataset[idx]
#     print(f"Test Sample {idx + 1} - Sample: {sample}")

train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False)

embedding_model = BARTPhoEmbedding(pretrained_model="vinai/bartpho-syllable")
num_classes = len(set(y_train))
model = TextClassifier(embedding_model, num_classes)

# Define loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

train_and_evaluate(model, train_dataloader, test_dataloader, criterion, optimizer, num_epochs=2)

Epoch - 1


Epoch 1/2, Test Accuracy: 69.03%
Epoch - 2


Epoch 2/2, Test Accuracy: 69.03%
